In [1]:
'''
This notebook has all of the original python files from the "\AutomatePython" directory combined into one
As seen in "_RunAllLocalPys.ipynb" we can just call external .py file from the same notebook.
But we created this file just to condense everything while still seeing the code. 
'''

'\nThis notebook has all of the original python files from the "\\AutomatePython" directory combined into one\nAs seen in "_RunAllLocalPys.ipynb" we can just call external .py file from the same notebook.\nBut we created this file just to condense everything while still seeing the code. \n'

In [9]:
# =================================Section 1 =================================
# Section 1 Make a pivot table.
# ============================================================================
# Installed these for python on the machine
# Also had to make sure they were applied to pycharm
# pip install xlrd
# pip install pandas
# pip install openpyxl
import pandas as pd

# Read Excel File
df = pd.read_excel('supermarket_sales.xlsx')

# Select columns: 'Gender', 'Product line', 'Total'
# These are 3 of the columns in the original spreadsheet
'''
Gender	    Product line            Total
Female	    Health and beauty       548.9715
Female	    Electronic accessories  80.22
Male	    Home and lifestyle      340.5255
Male	    Health and beauty       489.048
'''
df = df[['Gender', 'Product line', 'Total']]

# Make pivot table
pivot_table = df.pivot_table(index='Gender', columns='Product line', values='Total', aggfunc='sum').round(0)
'''
This is what the Pivot table will come out as, though I shortened up the column titles for notes
From original sheet, the gender column has either male and female, for thousands of purchases
So 'index' will separate those options from that specific collected column
Then 'columns' will separate the items in that specific collected column
Then 'values' along with 'sum' if a Male bought EA, add it to that correlated cell. 
---------------------------------------------------
Gender	EA	    FA	    F&B	    H&B	    H&L	    S&T
Female	27102	30437	33171	18561	30037	28575
Male	27236	23868	22974	30633	23825	26548

'''

# Export pivot table to Excel file
pivot_table.to_excel('1.make-pivot-table.xlsx', 'Report', startrow=4)


In [10]:
# =================================Section 2 =================================
# Section 2 add a Chart
# ============================================================================
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Reference

# Read workbook and select sheet
wb = load_workbook('1.make-pivot-table.xlsx')
sheet = wb['Report']

# Active rows and columns
min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

# Instantiate a barchart
barchart = BarChart()

# Locate data and categories
data = Reference(sheet,
                 min_col=min_column+1,
                 max_col=max_column,
                 min_row=min_row,
                 max_row=max_row)  # including headers

categories = Reference(sheet,
                       min_col=min_column,
                       max_col=min_column,
                       min_row=min_row+1,
                       max_row=max_row)  # not including headers

# Adding data and categories
barchart.add_data(data, titles_from_data=True)
barchart.set_categories(categories)

# Make chart
sheet.add_chart(barchart, "B12")
barchart.title = 'Sales by Product line'
barchart.style = 5  # choose the chart style

# Save workbook
wb.save('2.add-charts-barchart.xlsx')


In [11]:
# =================================Section 3 =================================
# Section 3 - Apply formulas
# ============================================================================
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

wb = load_workbook('2.add-charts-barchart.xlsx')
sheet = wb['Report']

min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

# Write an Excel formula with Python
# sheet['B8'] = '=SUM(B6:B7)'
# sheet['B8'].style = 'Currency'

# Write multiple formulas with a for loop
for i in range(min_column+1, max_column+1):  # (B, G+1)
    letter = get_column_letter(i)
    sheet[f'{letter}{max_row + 1}'] = f'=SUM({letter}{min_row + 1}:{letter}{max_row})'
    sheet[f'{letter}{max_row + 1}'].style = 'Currency'

wb.save('3.apply-formulas-report.xlsx')


In [12]:
# =================================Section 4 =================================
# Section 4 Format the cells
# ============================================================================
from openpyxl import load_workbook
from openpyxl.styles import Font

wb = load_workbook('3.apply-formulas-report.xlsx')
sheet = wb['Report']

# Add format
sheet['A1'] = 'Sales Report'
sheet['A2'] = 'January'
sheet['A1'].font = Font('Arial', bold=True, size=20)
sheet['A2'].font = Font('Arial', bold=True, size=10)

wb.save('4.report_january.xlsx')


In [8]:
# =================================Section 5 =================================
# Section 5 Pivot table to report
# ============================================================================
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Reference
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font

# Putting together #2, #3, and #4 (input: pivot_table.xlsx + month , output: Report with barchart, formulas and format)
month = 'february'

# Read workbook and select sheet
wb = load_workbook('1.make-pivot-table.xlsx')
sheet = wb['Report']

# Active rows and columns
min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

# Instantiate a barchart
barchart = BarChart()

# Locate data and categories
data = Reference(sheet, min_col=min_column+1, max_col=max_column, min_row=min_row, max_row=max_row)  # including headers
categories = Reference(sheet, min_col=min_column, max_col=min_column, min_row=min_row+1, max_row=max_row)  # not including headers

# Adding data and categories
barchart.add_data(data, titles_from_data=True)
barchart.set_categories(categories)

# Make chart
sheet.add_chart(barchart, "B12")
barchart.title = 'Sales by Product line'
barchart.style = 5  # choose the chart style

# Write multiple formulas with a for loop
for i in range(min_column+1, max_column+1):  # (B, G+1)
    letter = get_column_letter(i)
    sheet[f'{letter}{max_row + 1}'] = f'=SUM({letter}{min_row + 1}:{letter}{max_row})'
    sheet[f'{letter}{max_row + 1}'].style = 'Currency'

# Add format
sheet['A1'] = 'Sales Report'
sheet['A2'] = month
sheet['A1'].font = Font('Arial', bold=True, size=20)
sheet['A2'].font = Font('Arial', bold=True, size=10)

wb.save(f'5.report_{month}.xlsx')


In [7]:
# =================================Section 6 =================================
# Section 6 - py to exe
# ============================================================================
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Reference
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font
import os
import sys

# Preparing script before we convert it to executable
application_path = os.path.dirname(sys.executable)

# Putting together #2, #3, and #4 (input: pivot_table.xlsx + month , output: Report with barchart, formulas and format)
month = input('Introduce month: ')

# Read workbook and select sheet
wb = load_workbook('1.make-pivot-table.xlsx')
sheet = wb['Report']

# Active rows and columns
min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

# Instantiate a barchart
barchart = BarChart()

# Locate data and categories
data = Reference(sheet,
                 min_col=min_column+1,
                 max_col=max_column,
                 min_row=min_row,
                 max_row=max_row)  # including headers
categories = Reference(sheet,
                       min_col=min_column,
                       max_col=min_column,
                       min_row=min_row+1,
                       max_row=max_row)  # not including headers

# Adding data and categories
barchart.add_data(data, titles_from_data=True)
barchart.set_categories(categories)

# Make chart
sheet.add_chart(barchart, "B12")
barchart.title = 'Sales by Product line'
barchart.style = 5  # choose the chart style

# Write multiple formulas with a for loop
for i in range(min_column+1, max_column+1):  # (B, G+1)
    letter = get_column_letter(i)
    sheet[f'{letter}{max_row + 1}'] = f'=SUM({letter}{min_row + 1}:{letter}{max_row})'
    sheet[f'{letter}{max_row + 1}'].style = 'Currency'

# Add format
sheet['A1'] = 'Sales Report'
sheet['A2'] = month
sheet['A1'].font = Font('Arial', bold=True, size=20)
sheet['A2'].font = Font('Arial', bold=True, size=10)


wb.save(f'6.report_{month}.xlsx')


Introduce month:  June
